In [2]:
datos=spark\
    .read.csv("SMSSpamCollection",inferSchema=True,sep="\t")

In [3]:
datos.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [4]:
datos.count()

5574

In [6]:
datos.where("_c0='spam'").count()

747

In [7]:
datos.where("_c0='ham'").count()

4827

In [8]:
from pyspark.sql.functions import length

In [9]:
datos2=datos.withColumn("longitud",length("_c1"))

In [10]:
datos2.show()

+----+--------------------+--------+
| _c0|                 _c1|longitud|
+----+--------------------+--------+
| ham|Go until jurong p...|     111|
| ham|Ok lar... Joking ...|      29|
|spam|Free entry in 2 a...|     155|
| ham|U dun say so earl...|      49|
| ham|Nah I don't think...|      61|
|spam|FreeMsg Hey there...|     147|
| ham|Even my brother i...|      77|
| ham|As per your reque...|     160|
|spam|WINNER!! As a val...|     157|
|spam|Had your mobile 1...|     154|
| ham|I'm gonna be home...|     109|
|spam|SIX chances to wi...|     136|
|spam|URGENT! You have ...|     155|
| ham|I've been searchi...|     196|
| ham|I HAVE A DATE ON ...|      35|
|spam|XXXMobileMovieClu...|     149|
| ham|Oh k...i'm watchi...|      26|
| ham|Eh u remember how...|      81|
| ham|Fine if thats th...|      56|
|spam|England v Macedon...|     155|
+----+--------------------+--------+
only showing top 20 rows



In [11]:
datos2.groupBy("_c0").mean("longitud").show()

+----+-----------------+
| _c0|    avg(longitud)|
+----+-----------------+
| ham|71.45431945307645|
|spam|138.6706827309237|
+----+-----------------+



In [12]:
datos2.groupBy("_c0").min("longitud").show()

+----+-------------+
| _c0|min(longitud)|
+----+-------------+
| ham|            2|
|spam|           13|
+----+-------------+



In [13]:
datos2.groupBy("_c0").max("longitud").show()

+----+-------------+
| _c0|max(longitud)|
+----+-------------+
| ham|          910|
|spam|          223|
+----+-------------+



In [14]:
datos2.show()

+----+--------------------+--------+
| _c0|                 _c1|longitud|
+----+--------------------+--------+
| ham|Go until jurong p...|     111|
| ham|Ok lar... Joking ...|      29|
|spam|Free entry in 2 a...|     155|
| ham|U dun say so earl...|      49|
| ham|Nah I don't think...|      61|
|spam|FreeMsg Hey there...|     147|
| ham|Even my brother i...|      77|
| ham|As per your reque...|     160|
|spam|WINNER!! As a val...|     157|
|spam|Had your mobile 1...|     154|
| ham|I'm gonna be home...|     109|
|spam|SIX chances to wi...|     136|
|spam|URGENT! You have ...|     155|
| ham|I've been searchi...|     196|
| ham|I HAVE A DATE ON ...|      35|
|spam|XXXMobileMovieClu...|     149|
| ham|Oh k...i'm watchi...|      26|
| ham|Eh u remember how...|      81|
| ham|Fine if thats th...|      56|
|spam|England v Macedon...|     155|
+----+--------------------+--------+
only showing top 20 rows



In [15]:
from pyspark.ml.feature import (StringIndexer,VectorAssembler,
        RegexTokenizer,StopWordsRemover,CountVectorizer,IDF)

In [20]:
indexador=StringIndexer(inputCol="_c0",outputCol="label")
datosI=indexador.fit(datos2).transform(datos2).drop("_c0")

In [21]:
datosI.show()

+--------------------+--------+-----+
|                 _c1|longitud|label|
+--------------------+--------+-----+
|Go until jurong p...|     111|  0.0|
|Ok lar... Joking ...|      29|  0.0|
|Free entry in 2 a...|     155|  1.0|
|U dun say so earl...|      49|  0.0|
|Nah I don't think...|      61|  0.0|
|FreeMsg Hey there...|     147|  1.0|
|Even my brother i...|      77|  0.0|
|As per your reque...|     160|  0.0|
|WINNER!! As a val...|     157|  1.0|
|Had your mobile 1...|     154|  1.0|
|I'm gonna be home...|     109|  0.0|
|SIX chances to wi...|     136|  1.0|
|URGENT! You have ...|     155|  1.0|
|I've been searchi...|     196|  0.0|
|I HAVE A DATE ON ...|      35|  0.0|
|XXXMobileMovieClu...|     149|  1.0|
|Oh k...i'm watchi...|      26|  0.0|
|Eh u remember how...|      81|  0.0|
|Fine if thats th...|      56|  0.0|
|England v Macedon...|     155|  1.0|
+--------------------+--------+-----+
only showing top 20 rows



In [22]:
reTokenizer=RegexTokenizer(inputCol="_c1",
                           outputCol="palabras",
                          pattern="[ \,\.\?\!\;\(\)]+")

In [23]:
datosT=reTokenizer.transform(datosI)

In [24]:
datosT.show()

+--------------------+--------+-----+--------------------+
|                 _c1|longitud|label|            palabras|
+--------------------+--------+-----+--------------------+
|Go until jurong p...|     111|  0.0|[go, until, juron...|
|Ok lar... Joking ...|      29|  0.0|[ok, lar, joking,...|
|Free entry in 2 a...|     155|  1.0|[free, entry, in,...|
|U dun say so earl...|      49|  0.0|[u, dun, say, so,...|
|Nah I don't think...|      61|  0.0|[nah, i, don't, t...|
|FreeMsg Hey there...|     147|  1.0|[freemsg, hey, th...|
|Even my brother i...|      77|  0.0|[even, my, brothe...|
|As per your reque...|     160|  0.0|[as, per, your, r...|
|WINNER!! As a val...|     157|  1.0|[winner, as, a, v...|
|Had your mobile 1...|     154|  1.0|[had, your, mobil...|
|I'm gonna be home...|     109|  0.0|[i'm, gonna, be, ...|
|SIX chances to wi...|     136|  1.0|[six, chances, to...|
|URGENT! You have ...|     155|  1.0|[urgent, you, hav...|
|I've been searchi...|     196|  0.0|[i've, been, sear..

In [28]:
StopWordsRemover().loadDefaultStopWords("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 '

In [29]:
filtro=StopWordsRemover(inputCol="palabras",
                        outputCol="filtradas")

In [34]:
datosF=filtro.transform(datosT).drop("_c1","palabras")

In [35]:
datosF.show()

+--------+-----+--------------------+
|longitud|label|           filtradas|
+--------+-----+--------------------+
|     111|  0.0|[go, jurong, poin...|
|      29|  0.0|[ok, lar, joking,...|
|     155|  1.0|[free, entry, 2, ...|
|      49|  0.0|[u, dun, say, ear...|
|      61|  0.0|[nah, think, goes...|
|     147|  1.0|[freemsg, hey, da...|
|      77|  0.0|[even, brother, l...|
|     160|  0.0|[per, request, 'm...|
|     157|  1.0|[winner, valued, ...|
|     154|  1.0|[mobile, 11, mont...|
|     109|  0.0|[gonna, home, soo...|
|     136|  1.0|[six, chances, wi...|
|     155|  1.0|[urgent, won, 1, ...|
|     196|  0.0|[searching, right...|
|      35|  0.0|      [date, sunday]|
|     149|  1.0|[xxxmobilemoviecl...|
|      26|  0.0|[oh, k, watching,...|
|      81|  0.0|[eh, u, remember,...|
|      56|  0.0|[fine, thats, wa...|
|     155|  1.0|[england, v, mace...|
+--------+-----+--------------------+
only showing top 20 rows



In [36]:
cv=CountVectorizer(inputCol="filtradas",outputCol="cv")

In [39]:
datoscv=cv.fit(datosF).transform(datosF)

In [42]:
datoscv.show(1)

+--------+-----+--------------------+--------------------+
|longitud|label|           filtradas|                  cv|
+--------+-----+--------------------+--------------------+
|     111|  0.0|[go, jurong, poin...|(9493,[8,14,35,55...|
+--------+-----+--------------------+--------------------+
only showing top 1 row



In [43]:
idf=IDF(inputCol="cv",outputCol="idf")

In [44]:
datosIdf=idf.fit(datoscv).transform(datoscv)

In [48]:
datosIdf.select("cv","idf").show(1,vertical=True,truncate=False)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 cv  | (9493,[8,14,35,55,71,85,316,530,661,759,1232,1272,1288,2962,7205,7924],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                                                                                                                                                                                                                   
 idf | (9493,[8,14,35,55,71,85,316,530,661,759,1232,1272,1288,2962,7205,7924],[3.0500984931820034,3.19232559277408,3.7977338590260183,3.934699714099176,4.061699404860483,4.256599743861298,5.224850214666164,5.681608617161879,5.986990266713061,5.986990266713

In [49]:
datosIdf.printSchema()

root
 |-- longitud: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- filtradas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- cv: vector (nullable = true)
 |-- idf: vector (nullable = true)



In [51]:
constructor=VectorAssembler(inputCols=["longitud","idf"],
                            outputCol="features")

datosNuevos=constructor.transform(datosIdf)

In [52]:
datosNuevos.show()

+--------+-----+--------------------+--------------------+--------------------+--------------------+
|longitud|label|           filtradas|                  cv|                 idf|            features|
+--------+-----+--------------------+--------------------+--------------------+--------------------+
|     111|  0.0|[go, jurong, poin...|(9493,[8,14,35,55...|(9493,[8,14,35,55...|(9494,[0,9,15,36,...|
|      29|  0.0|[ok, lar, joking,...|(9493,[0,6,229,35...|(9493,[0,6,229,35...|(9494,[0,1,7,230,...|
|     155|  1.0|[free, entry, 2, ...|(9493,[2,10,21,31...|(9493,[2,10,21,31...|(9494,[0,3,11,22,...|
|      49|  0.0|[u, dun, say, ear...|(9493,[0,76,78,87...|(9493,[0,76,78,87...|(9494,[0,1,77,79,...|
|      61|  0.0|[nah, think, goes...|(9493,[46,131,352...|(9493,[46,131,352...|(9494,[0,47,132,3...|
|     147|  1.0|[freemsg, hey, da...|(9493,[6,12,19,30...|(9493,[6,12,19,30...|(9494,[0,7,13,20,...|
|      77|  0.0|[even, brother, l...|(9493,[12,122,290...|(9493,[12,122,290...|(9494,[0,13,

In [55]:
from pyspark.ml.classification import (LogisticRegression,
                                       RandomForestClassifier)
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

In [56]:
entrena,evalua=datosNuevos.randomSplit([0.8,0.2])

In [57]:
lr=LogisticRegression()
rf=RandomForestClassifier(numTrees=100)

In [59]:
modeloLr=lr.fit(entrena)
modelorRf=rf.fit(entrena)

In [61]:
predLr=modeloLr.transform(evalua)
predRf=modelorRf.transform(evalua)

In [62]:
evaluadorB=BinaryClassificationEvaluator()
evaluadorM=MulticlassClassificationEvaluator(metricName="accuracy")

In [63]:
evaluadorB.evaluate(predLr),evaluadorM.evaluate(predLr)

(0.9695050190377248, 0.97214734950584)

In [64]:
evaluadorB.evaluate(predRf),evaluadorM.evaluate(predRf)

(0.9860782277604706, 0.8661275831087152)

In [66]:
predLr.select("label","prediction").show(100)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|     